In [ ]:
# Set up paths
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import Config and Main Class
from src.personalai_main import PersonalAI, PersonalAIConfig
from src.pipelines.qa.qa_engine import QAEngine  # Import the robust engine directly

# Initialize PersonalAI (to load KG Model)
config = PersonalAIConfig()
config.verbose = True 
pai = PersonalAI(config)

# Initialize QAEngine using the loaded KG Model
# We use the standard E5 model path or name
model_path = "intfloat/multilingual-e5-small"
qa_engine = QAEngine(pai.kg_model, finetuned_model_path=model_path)

# Handy wrapper function using QAEngine
def ask(question: str):
    print(f"Question: {question}")
    # Get ranked results from QAEngine
    results = qa_engine.get_ranked_results(question, top_k=3)
    
    if results:
        top_res = results[0]
        print(f"Answer: {top_res['text']}")
        print(f"Confidence: {top_res['confidence']:.4f}")
        if top_res.get('temporal_score'):
            print(f"Temporal Score: {top_res['temporal_score']}")
    else:
        print("Answer: No results found.")
    print("-" * 50)

✓ Используется Apple Silicon GPU (MPS)
Query failed: {neo4j_code: Neo.ClientError.Statement.UnsupportedAdministrationCommand} {message: Unsupported administration command: CREATE DATABASE neo4j IF NOT EXISTS} {gql_status: 50N42} {gql_status_description: error: general processing exception - unexpected error. Unsupported administration command: CREATE DATABASE neo4j IF NOT EXISTS}
Error query:  CREATE DATABASE neo4j IF NOT EXISTS


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: intfloat/multilingual-e5-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model path intfloat/multilingual-e5-small not found locally, assuming HuggingFace model name.
✓ Используется Apple Silicon GPU (MPS)


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: intfloat/multilingual-e5-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


DEBUG: [QAEngine] Initializing WikidataMapper with path: /Users/nmuravya/Desktop/KG_sber/Personal-AI-dev 2/wikidata_big/kg
✓ Используется Apple Silicon GPU (MPS)
Loading TemporalScorer resources from /Users/nmuravya/Desktop/KG_sber/Personal-AI-dev 2/wikidata_big/kg/tkbc_processed_data/wikidata_big/...
Loaded mappings: 125726 entities, 203 relations, 9621 timestamps
Loading weights from /Users/nmuravya/Desktop/KG_sber/Personal-AI-dev 2/models/cronkgqa/tcomplex.ckpt...
TemporalScorer initialized successfully.


# QA Experiments: 7 Question Types (v2 - Direct Engine)

We test the system on specific question types using the **QAEngine** directly to ensure robust temporal scoring:
1.  **Simple Entity**: Fact retrieval (Who/What).
2.  **Simple Time**: Fact retrieval (When).
3.  **Before**: Temporal constraint.
4.  **After**: Temporal constraint.
5.  **First/Last**: Ordinal reasoning.
6.  **Time Join**: Event correlation (Who was X when Y happened).
7.  **Relation**: Identifying relationships between entities.

In [10]:
print("\n=== Type 1: Simple Entity Question ===")
# Expected Answer: Vera Nabokova (spouse)
# Logic: Direct retrieval of the 'spouse' relation for 'Vladimir Nabokov'.
ask("Who was the spouse of Vladimir Nabokov?")


=== Type 1: Simple Entity Question ===
Question: Who was the spouse of Vladimir Nabokov?
DEBUG: [Ollama] Entities: ['Vladimir Nabokov'], Time: None, Relation: spouse
DEBUG: [Mapper] 'Vladimir Nabokov' -> Wikidata ID: Q36591
DEBUG: [Graph Match (Neo4j)] Found node: Vladimir Nabokov (ID: ce39fe19182d14873fcf7c943767cb09)
DEBUG: [QAEngine] Navigating from IDs: ['Q36591']
DEBUG: [KGNavigator] Getting neighborhood for ['Q36591'] at depth 1
DEBUG: [KGNavigator] Node Q36591 has 15 adjacent nodes
DEBUG: [KGNavigator] Total quadruplets found: 23
Answer: 1925 - 1977: Vera Nabokova spouse Vladimir Nabokov
Confidence: 1.0000
Temporal Score: 1.00 (Logit: 15.63)
--------------------------------------------------


In [11]:
print("\n=== Type 2: Simple Time Question ===")
# Expected Answer: 1899 (date of birth)
# Logic: Direct retrieval of the 'date of birth' relation/time property.
ask("When was Vladimir Nabokov born?")


=== Type 2: Simple Time Question ===
Question: When was Vladimir Nabokov born?
DEBUG: [Ollama] Entities: ['Vladimir Nabokov'], Time: 1899, Relation: born
DEBUG: [Mapper] 'Vladimir Nabokov' -> Wikidata ID: Q36591
DEBUG: [Graph Match (Neo4j)] Found node: Vladimir Nabokov (ID: ce39fe19182d14873fcf7c943767cb09)
DEBUG: [QAEngine] Navigating from IDs: ['Q36591']
DEBUG: [KGNavigator] Getting neighborhood for ['Q36591'] at depth 1
DEBUG: [KGNavigator] Node Q36591 has 15 adjacent nodes
DEBUG: [KGNavigator] Total quadruplets found: 23
Answer: 1922 - 1937: Vladimir Nabokov residence Berlin
Confidence: 0.9170
Temporal Score: 1.00 (Logit: 8.96)
--------------------------------------------------


In [12]:
print("\n=== Type 3: Temporal Constraint (Before) ===")
# Expected Answer: Saint Petersburg (lived 1899-1917) or Cambridge (1919-1922)
# Logic: Retrieve 'residence' facts and filter/score based on time < 1930.
ask("Where did Vladimir Nabokov live before 1930?")


=== Type 3: Temporal Constraint (Before) ===
Question: Where did Vladimir Nabokov live before 1930?
DEBUG: [Ollama] Entities: ['Russia', 'Vladimir Nabokov'], Time: null, Relation: lived in
DEBUG: [Mapper] 'Russia' -> Wikidata ID: Q159
DEBUG: [Mapper] 'Vladimir Nabokov' -> Wikidata ID: Q36591
DEBUG: [Graph Match (Neo4j)] Found node: Russia (ID: 5feb168ca8fb495dcc89b1208cdeb919)
DEBUG: [Graph Match (Neo4j)] Found node: Vladimir Nabokov (ID: ce39fe19182d14873fcf7c943767cb09)
DEBUG: [QAEngine] Navigating from IDs: ['Q159', 'Q36591']
DEBUG: [KGNavigator] Getting neighborhood for ['Q159', 'Q36591'] at depth 1
DEBUG: [KGNavigator] Node Q159 has 61 adjacent nodes
DEBUG: [KGNavigator] Node Q36591 has 15 adjacent nodes
DEBUG: [KGNavigator] Total quadruplets found: 110
Answer: 1922 - 1937: Vladimir Nabokov residence Berlin
Confidence: 0.9068
Temporal Score: None (Unknown to Scorer)
--------------------------------------------------


In [13]:
print("\n=== Type 4: Temporal Constraint (After) ===")
# Expected Answer: Montreux (lived 1961-1977)
# Logic: Retrieve 'residence' facts and filter/score based on time > 1960.
ask("Where did Vladimir Nabokov live after 1960?")


=== Type 4: Temporal Constraint (After) ===
Question: Where did Vladimir Nabokov live after 1960?
DEBUG: [Ollama] Entities: ['Vladimir Nabokov'], Time: 1960, Relation: lived in
DEBUG: [Mapper] 'Vladimir Nabokov' -> Wikidata ID: Q36591
DEBUG: [Graph Match (Neo4j)] Found node: Vladimir Nabokov (ID: ce39fe19182d14873fcf7c943767cb09)
DEBUG: [QAEngine] Navigating from IDs: ['Q36591']
DEBUG: [KGNavigator] Getting neighborhood for ['Q36591'] at depth 1
DEBUG: [KGNavigator] Node Q36591 has 15 adjacent nodes
DEBUG: [KGNavigator] Total quadruplets found: 23
Answer: 1961 - 1977: Vladimir Nabokov residence Montreux
Confidence: 0.9223
Temporal Score: 1.00 (Logit: 11.96)
--------------------------------------------------


In [14]:
print("\n=== Type 5: First/Last Question (Ordinal) ===")
# Expected Answer: Mashenka (Mary) - 1926 (First novel)
# Logic: Retrieve all 'notable works' or 'novels' and select the one with the earliest time.
ask("What was the first novel of Vladimir Nabokov?")


=== Type 5: First/Last Question (Ordinal) ===
Question: What was the first novel of Vladimir Nabokov?
DEBUG: [Ollama] Entities: ['novel', 'Vladimir Nabokov'], Time: None, Relation: first
DEBUG: [Mapper] 'novel' -> Wikidata ID: None
DEBUG: [Mapper] 'Vladimir Nabokov' -> Wikidata ID: Q36591
DEBUG: [Graph Match (Neo4j)] Found node: Vladimir Nabokov (ID: ce39fe19182d14873fcf7c943767cb09)
DEBUG: [QAEngine] Navigating from IDs: ['Q36591']
DEBUG: [KGNavigator] Getting neighborhood for ['Q36591'] at depth 1
DEBUG: [KGNavigator] Node Q36591 has 15 adjacent nodes
DEBUG: [KGNavigator] Total quadruplets found: 23
Answer: 1965 - 1965: Vladimir Nabokov nominated for Nobel Prize in Literature
Confidence: 0.9079
Temporal Score: 1.00 (Logit: 12.32)
--------------------------------------------------


In [15]:
print("\n=== Type 6: Time Join Question ===")
# Expected Answer: William McKinley (President of USA in 1899)
# Logic: Get Nabokov's birth year (1899) -> Find who was President of USA in 1899.
# This tests multi-hop temporal reasoning.
ask("Who was the president of USA when Vladimir Nabokov was born?")


=== Type 6: Time Join Question ===
Question: Who was the president of USA when Vladimir Nabokov was born?
DEBUG: [Ollama] Entities: ['USA', 'Vladimir Nabokov'], Time: 1899, Relation: president
DEBUG: [Mapper] 'USA' -> Wikidata ID: None
DEBUG: [Mapper] 'Vladimir Nabokov' -> Wikidata ID: Q36591
DEBUG: [Graph Match (Neo4j)] Found node: Vladimir Nabokov (ID: ce39fe19182d14873fcf7c943767cb09)
DEBUG: [QAEngine] Navigating from IDs: ['Q36591']
DEBUG: [KGNavigator] Getting neighborhood for ['Q36591'] at depth 1
DEBUG: [KGNavigator] Node Q36591 has 15 adjacent nodes
DEBUG: [KGNavigator] Total quadruplets found: 23
Answer: 1940 - 1941: Vladimir Nabokov residence Manhattan
Confidence: 0.8965
Temporal Score: 1.00 (Logit: 7.18)
--------------------------------------------------


In [16]:
print("\n=== Type 7: Relation Question ===")
# Expected Answer: spouse
# Logic: Identify the relation connecting two given entities.
ask("What is the relation between Vladimir Nabokov and Vera Nabokova?")


=== Type 7: Relation Question ===
Question: What is the relation between Vladimir Nabokov and Vera Nabokova?
DEBUG: [Ollama] Entities: ['Vera Nabokova', 'Vladimir Nabokov'], Time: None, Relation: spouse
DEBUG: [Mapper] 'Vera Nabokova' -> Wikidata ID: Q4311259
DEBUG: [Mapper] 'Vladimir Nabokov' -> Wikidata ID: Q36591
DEBUG: [Graph Match (Neo4j)] Found node: Vera Nabokova (ID: c7fb96120705aeb38af96cdf53fd4e9c)
DEBUG: [Graph Match (Neo4j)] Found node: Vladimir Nabokov (ID: ce39fe19182d14873fcf7c943767cb09)
DEBUG: [QAEngine] Navigating from IDs: ['Q4311259', 'Q36591']
DEBUG: [KGNavigator] Getting neighborhood for ['Q4311259', 'Q36591'] at depth 1
DEBUG: [KGNavigator] Node Q36591 has 15 adjacent nodes
DEBUG: [KGNavigator] Node Q4311259 has 2 adjacent nodes
DEBUG: [KGNavigator] Total quadruplets found: 27
Answer: 1925 - 1977: Vera Nabokova spouse Vladimir Nabokov
Confidence: 1.0000
Temporal Score: 1.00 (Logit: 15.63)
--------------------------------------------------
